In [2]:
%cd ../..

D:\Programming\Kaggle\Studcamp\fake-reviews-detection-app


In [16]:
import pandas as pd
import numpy as np

from wordcloud import WordCloud

from typing import Callable

import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords

import textstat
from razdel import tokenize

from tqdm.notebook import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=12, progress_bar=True)

from sklearn.feature_extraction.text import TfidfVectorizer

from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split

import fasttext

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
from src.preprocessing.clean_text import clean_text

In [5]:
def clean_text(text):
    from razdel import tokenize
    from pymorphy3 import MorphAnalyzer
    from nltk.corpus import stopwords

    text = text.lower()

    sw_to_drop = ['не']
    
    morph = MorphAnalyzer()
    sws = set(stopwords.words('russian'))

    for sw in sw_to_drop:
        sws.remove(sw)
    
    tokens = tokenize(text)
    tokens = [morph.parse(token.text)[0].normal_form for token in tokens]
    tokens = [token for token in tokens if token not in sws]

    return ' '.join(tokens) 

In [7]:
reviews = pd.read_csv('./src/text_eda/reviews_preprocessed_with_textstat.csv')
reviews

,is_AI,review,processed_text,automated_readability_index,avg_letter_per_word,avg_sentence_length,avg_sentence_per_word,char_count,coleman_liau_index,crawford,...,linsear_write_formula,lix,long_word_count,mcalpine_eflaw,miniword_count,monosyllabcount,reading_time,sentence_count,syllable_count,words_per_sentence
0,0,"Продавец выслал не тот цвет, я очень расстроил...","продавец выслать не цвет , очень расстроиться ...",5.6,4.86,6.0,0.17,184,7.36,-2.0,...,2.000000,36.56,7,8.5,15,36,2.70,6,36,6.000000
1,1,Отличные часы! Купил их для подруги. Качество ...,отличный часы ! купить подруга . качество отли...,15.4,6.71,7.0,0.14,99,18.97,-1.5,...,2.500000,71.29,9,8.0,2,14,1.45,2,14,7.000000
2,0,редкостная дрянь....,редкостный дрянь ....,24.3,7.50,2.0,0.50,19,12.90,-8.9,...,0.000000,102.00,1,2.0,0,2,0.28,1,2,2.000000
3,1,"Купил отцу этот фильтр, он доволен. Но я счита...","купить отец фильтр , довольный . считать , дос...",7.7,5.00,9.0,0.11,94,9.94,-0.9,...,3.500000,42.33,4,11.5,5,18,1.38,2,18,9.000000
4,1,Купил подгузники Huggies Elite Soft для моей с...,"купить подгузник huggies elite soft свекровь ,...",9.9,5.48,8.3,0.12,144,12.43,-0.9,...,3.166667,48.30,10,11.0,8,24,2.12,3,26,8.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29283,1,Купил витамины для волос после рекомендации тр...,купить витамин волос рекомендация трихолог . в...,12.2,6.04,8.7,0.12,162,15.68,-0.8,...,3.333333,43.32,9,10.3,5,25,2.38,3,27,8.666667
29284,1,Покрывало качественное. Мягкое и приятное на о...,покрывало качественный . мягкий приятный ощупь...,13.1,6.27,7.5,0.13,98,16.72,-1.3,...,2.750000,54.17,7,9.5,4,15,1.44,2,15,7.500000
29285,1,Коллега будет доволен. Вещь хорошая\n,коллега довольный . вещь хороший,10.3,6.00,5.0,0.20,31,13.08,-2.7,...,1.500000,65.00,3,5.0,0,5,0.46,1,5,5.000000
29286,0,ужасные шорты. в реальности сильно отличаются ...,ужасный шорты . реальность сильно отличаться ш...,9.7,5.25,9.2,0.11,310,11.39,-0.8,...,3.666667,44.91,18,12.2,18,55,4.55,6,55,9.166667


In [9]:
train, test = train_test_split(reviews, test_size=0.2, random_state=42)

In [40]:
with open('train.txt', 'wt', encoding='UTF-8') as f:
    for k in range(train['review'].shape[0]):
        f.write(f"__label__{train['is_AI'].iloc[k]} {train['review'].iloc[k]}\n")
with open('eval.txt', 'wt', encoding='UTF-8') as f:
    for k in range(test['review'].shape[0]):
        f.write(f"__label__{test['is_AI'].iloc[k]} {test['review'].iloc[k]}\n")

model = fasttext.train_supervised(input="train.txt", neg=5, epoch=20)

y_pred = []
for k in range(test.shape[0]):
    y_pred.append(int(model.predict(test.iloc[k]['review'].replace('\n', ' '))[0][0][9:]))

print('f1_score', f1_score(test['is_AI'], y_pred))
print(classification_report(test['is_AI'], y_pred))

f1_score 0.9901264507188636
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2971
           1       0.99      0.99      0.99      2887

    accuracy                           0.99      5858
   macro avg       0.99      0.99      0.99      5858
weighted avg       0.99      0.99      0.99      5858



In [41]:
with open('train.txt', 'wt', encoding='UTF-8') as f:
    for k in range(train['processed_text'].shape[0]):
        f.write(f"__label__{train['is_AI'].iloc[k]} {train['processed_text'].iloc[k]}\n")
with open('eval.txt', 'wt', encoding='UTF-8') as f:
    for k in range(test['processed_text'].shape[0]):
        f.write(f"__label__{test['is_AI'].iloc[k]} {test['processed_text'].iloc[k]}\n")

model = fasttext.train_supervised(input="train.txt", neg=5, epoch=20)

y_pred = []
for k in range(test.shape[0]):
    y_pred.append(int(model.predict(test.iloc[k]['processed_text'].replace('\n', ' '))[0][0][9:]))

print('f1_score', f1_score(test['is_AI'], y_pred))
print(classification_report(test['is_AI'], y_pred))

f1_score 0.9905057828413603
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2971
           1       0.99      0.99      0.99      2887

    accuracy                           0.99      5858
   macro avg       0.99      0.99      0.99      5858
weighted avg       0.99      0.99      0.99      5858



In [9]:
veccer = TfidfVectorizer().fit(reviews['processed_review'])
train_vec = veccer.transform(reviews['processed_review']).toarray()
test_vec = veccer.transform(test_reviews['processed_review']).toarray()

In [14]:
model = LogisticRegression().fit(train_vec, reviews['is_AI'])
print(classification_report(test_reviews['is_AI'], model.predict(test_vec)))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       869
           1       0.98      0.96      0.97       890

    accuracy                           0.97      1759
   macro avg       0.97      0.97      0.97      1759
weighted avg       0.97      0.97      0.97      1759



In [15]:
model = CatBoostClassifier().fit(train_vec, reviews['is_AI'])
print(classification_report(test_reviews['is_AI'], model.predict(test_vec)))

Learning rate set to 0.027499
0:	learn: 0.6740608	total: 262ms	remaining: 4m 21s
1:	learn: 0.6567224	total: 351ms	remaining: 2m 54s
2:	learn: 0.6412283	total: 438ms	remaining: 2m 25s
3:	learn: 0.6248145	total: 520ms	remaining: 2m 9s
4:	learn: 0.6095905	total: 600ms	remaining: 1m 59s
5:	learn: 0.5959138	total: 683ms	remaining: 1m 53s
6:	learn: 0.5829130	total: 765ms	remaining: 1m 48s
7:	learn: 0.5707686	total: 847ms	remaining: 1m 45s
8:	learn: 0.5595528	total: 928ms	remaining: 1m 42s
9:	learn: 0.5493098	total: 1.02s	remaining: 1m 40s
10:	learn: 0.5393887	total: 1.1s	remaining: 1m 39s
11:	learn: 0.5294914	total: 1.19s	remaining: 1m 37s
12:	learn: 0.5199513	total: 1.27s	remaining: 1m 36s
13:	learn: 0.5111011	total: 1.35s	remaining: 1m 35s
14:	learn: 0.5035079	total: 1.43s	remaining: 1m 34s
15:	learn: 0.4970748	total: 1.52s	remaining: 1m 33s
16:	learn: 0.4892774	total: 1.6s	remaining: 1m 32s
17:	learn: 0.4823229	total: 1.68s	remaining: 1m 31s
18:	learn: 0.4773210	total: 1.76s	remaining: 1m

In [21]:
model = CatBoostClassifier(text_features=['review']).fit(reviews.drop(columns=['processed_review']), reviews['is_AI'])
print(classification_report(test_reviews['is_AI'], model.predict(test_reviews.drop(columns=['processed_review']))))

Learning rate set to 0.027499
0:	learn: 0.5832559	total: 91.3ms	remaining: 1m 31s
1:	learn: 0.4885102	total: 192ms	remaining: 1m 36s
2:	learn: 0.4084464	total: 300ms	remaining: 1m 39s
3:	learn: 0.3413640	total: 413ms	remaining: 1m 42s
4:	learn: 0.2881039	total: 520ms	remaining: 1m 43s
5:	learn: 0.2402100	total: 626ms	remaining: 1m 43s
6:	learn: 0.2002642	total: 710ms	remaining: 1m 40s
7:	learn: 0.1697939	total: 794ms	remaining: 1m 38s
8:	learn: 0.1421136	total: 875ms	remaining: 1m 36s
9:	learn: 0.1190160	total: 951ms	remaining: 1m 34s
10:	learn: 0.0999757	total: 1.03s	remaining: 1m 32s
11:	learn: 0.0841930	total: 1.11s	remaining: 1m 31s
12:	learn: 0.0712986	total: 1.2s	remaining: 1m 31s
13:	learn: 0.0607999	total: 1.29s	remaining: 1m 30s
14:	learn: 0.0519181	total: 1.37s	remaining: 1m 30s
15:	learn: 0.0454070	total: 1.46s	remaining: 1m 29s
16:	learn: 0.0389092	total: 1.55s	remaining: 1m 29s
17:	learn: 0.0334910	total: 1.63s	remaining: 1m 29s
18:	learn: 0.0292648	total: 1.72s	remaining: